# Self-Model Visualization and Interaction Tools

This notebook provides interactive visualization tools for exploring the system's self-knowledge, capability boundaries, confidence levels, and temporal evolution of self-awareness.

## Table of Contents
1. [Setup and Configuration](#setup)
2. [Self-Knowledge Visualization](#self-knowledge)
3. [Capability Boundary Explorer](#capability-boundaries)
4. [Confidence Visualization](#confidence)
5. [Temporal Evolution Tracker](#temporal-evolution)
6. [Anomaly Detection](#anomaly-detection)
7. [Cross-Container Comparison](#cross-container)

## Setup and Configuration <a id="setup"></a>

In [ ]:
import os
import sys
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import seaborn as sns
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import networkx as nx
import warnings
warnings.filterwarnings('ignore')

# Add system path for importing custom modules
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

# Import the self-awareness framework
from system.ai_frameworks.self_awareness import SelfAwarenessFramework, AwarenessDimension

In [ ]:
# Initialize the Self-Awareness Framework
def initialize_framework():
    framework = SelfAwarenessFramework()
    framework.start()
    return framework

# Load self-model from file if it exists, otherwise initialize a new one
def load_self_model(filepath='../data/self_models/current_self_model.json'):
    if os.path.exists(filepath):
        with open(filepath, 'r') as f:
            return json.load(f)
    else:
        # No saved model, create a new framework and get its self-model
        framework = initialize_framework()
        model = framework.get_self_model()
        
        # Create directory if it doesn't exist
        os.makedirs(os.path.dirname(filepath), exist_ok=True)
        
        # Save model to file
        with open(filepath, 'w') as f:
            json.dump(model, f, indent=2)
            
        return model

# Function to load self-models from both containers
def load_container_models():
    models = {}
    
    # Look for self-models in both containers
    for container in ['terminal_1', 'terminal_2']:
        filepath = f'../data/self_models/{container}_self_model.json'
        if os.path.exists(filepath):
            with open(filepath, 'r') as f:
                models[container] = json.load(f)
    
    return models

# Load the current self-model
self_model = load_self_model()
print(f"Loaded self-model: {self_model['id']} (created at {datetime.fromtimestamp(self_model['created_at'])}")

## Self-Knowledge Visualization <a id="self-knowledge"></a>

This section provides tools for visualizing the system's self-knowledge, represented as a graph of knowledge elements, capabilities, and their relationships.

In [ ]:
def create_knowledge_graph(self_model):
    """Create a NetworkX graph from the self-model knowledge representation."""
    G = nx.DiGraph()
    
    # Add knowledge nodes and capability nodes
    knowledge_boundaries = self_model.get('knowledge_boundaries', [])
    capabilities = self_model.get('capabilities', {}).get('capabilities', {})
    
    # Add capability nodes
    for cap_id, cap_info in capabilities.items():
        color = 'green' if cap_info.get('enabled', True) else 'red'
        intensity = cap_info.get('performance', 0.5)
        G.add_node(cap_id, 
                  node_type='capability', 
                  description=cap_info.get('description', ''),
                  performance=cap_info.get('performance', 0),
                  confidence=cap_info.get('confidence', 0),
                  enabled=cap_info.get('enabled', True),
                  color=color,
                  size=30 + 20 * intensity)
    
    # Add knowledge boundary nodes
    for kb in knowledge_boundaries:
        G.add_node(kb, 
                  node_type='boundary', 
                  color='orange',
                  size=25)
        
        # Connect knowledge boundaries to related capabilities
        # This is a heuristic - in a real system, these connections would be explicit
        for cap_id in capabilities:
            if any(kb_part in cap_id for kb_part in kb.split('_')):
                G.add_edge(kb, cap_id, edge_type='influences')
    
    # Add basic system performance nodes
    if 'last_state' in self_model:
        state = self_model['last_state']
        G.add_node('system_memory', 
                  node_type='resource',
                  usage=state.get('memory_percent', 0),
                  color='blue',
                  size=20)
        G.add_node('system_cpu', 
                  node_type='resource',
                  usage=state.get('cpu_percent', 0),
                  color='blue',
                  size=20)
        
        # Connect resources to capabilities
        for cap_id in capabilities:
            G.add_edge('system_memory', cap_id, edge_type='supports')
            G.add_edge('system_cpu', cap_id, edge_type='supports')
    
    return G

def visualize_knowledge_graph(self_model):
    """Create an interactive visualization of the knowledge graph."""
    G = create_knowledge_graph(self_model)
    
    # Create positions using a force-directed layout
    pos = nx.spring_layout(G)
    
    # Create Plotly figure
    fig = go.Figure()
    
    # Add edges
    edge_x = []
    edge_y = []
    for edge in G.edges():
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])
    
    fig.add_trace(go.Scatter(
        x=edge_x, y=edge_y,
        line=dict(width=0.5, color='#888'),
        hoverinfo='none',
        mode='lines'))
    
    # Collect nodes by type for different styling
    node_types = {}
    for node, attrs in G.nodes(data=True):
        node_type = attrs.get('node_type', 'unknown')
        if node_type not in node_types:
            node_types[node_type] = {'nodes': [], 'x': [], 'y': [], 'text': [], 
                                     'color': [], 'size': []}
        
        node_types[node_type]['nodes'].append(node)
        node_types[node_type]['x'].append(pos[node][0])
        node_types[node_type]['y'].append(pos[node][1])
        
        text = f"<b>{node}</b><br>"
        for k, v in attrs.items():
            if k not in ['node_type', 'color', 'size']:
                text += f"{k}: {v}<br>"
        node_types[node_type]['text'].append(text)
        
        node_types[node_type]['color'].append(attrs.get('color', 'gray'))
        node_types[node_type]['size'].append(attrs.get('size', 15))
    
    # Add nodes by type
    for node_type, data in node_types.items():
        fig.add_trace(go.Scatter(
            x=data['x'],
            y=data['y'],
            mode='markers',
            name=node_type,
            marker=dict(
                size=data['size'],
                color=data['color'],
                line_width=2),
            text=data['text'],
            hoverinfo='text'))
    
    # Update layout
    fig.update_layout(
        title=f"System Self-Knowledge Visualization",
        titlefont_size=16,
        showlegend=True,
        hovermode='closest',
        margin=dict(b=20, l=5, r=5, t=40),
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        width=800,
        height=600,
        legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01)
    )
    
    return fig

In [ ]:
# Create an interactive knowledge graph visualization
knowledge_graph_fig = visualize_knowledge_graph(self_model)
knowledge_graph_fig.show()

## Capability Boundary Explorer <a id="capability-boundaries"></a>

This tool provides an interactive exploration of system capabilities and their boundaries, allowing users to understand what the system can and cannot do with confidence.

In [ ]:
def create_capability_radar_chart(self_model):
    """Create a radar chart showing system capabilities and their performance."""
    capabilities = self_model.get('capabilities', {}).get('capabilities', {})
    
    if not capabilities:
        return go.Figure().update_layout(title="No capability data available")
    
    # Extract capability data
    cap_names = list(capabilities.keys())
    performances = [capabilities[c].get('performance', 0) for c in cap_names]
    confidences = [capabilities[c].get('confidence', 0) for c in cap_names]
    
    # Create radar chart
    fig = go.Figure()
    
    fig.add_trace(go.Scatterpolar(
        r=performances,
        theta=cap_names,
        fill='toself',
        name='Performance',
        line=dict(color='teal')
    ))
    
    fig.add_trace(go.Scatterpolar(
        r=confidences,
        theta=cap_names,
        fill='toself',
        name='Confidence',
        line=dict(color='coral')
    ))
    
    # Update layout
    fig.update_layout(
        title="System Capabilities and Confidence",
        polar=dict(
            radialaxis=dict(
                visible=True,
                range=[0, 1]
            )),
        showlegend=True
    )
    
    return fig

def create_capability_details_ui(self_model):
    """Create an interactive UI for exploring capability details."""
    capabilities = self_model.get('capabilities', {}).get('capabilities', {})
    
    if not capabilities:
        return HTML("<p>No capability data available</p>")
    
    # Create dropdown for selecting capability
    dropdown = widgets.Dropdown(
        options=list(capabilities.keys()),
        description='Capability:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='50%')
    )
    
    # Output widget for displaying details
    output = widgets.Output()
    
    def update_details(change):
        with output:
            clear_output()
            capability = change['new']
            cap_data = capabilities[capability]
            
            # Create detail table
            html = f"<h3>Details for: {capability}</h3>"
            html += "<table style='width:80%; border-collapse: collapse;'>"
            
            # Add rows for each attribute
            for key, value in cap_data.items():
                html += f"<tr style='border-bottom: 1px solid #ddd;'>"
                html += f"<td style='padding: 8px; text-align: left; width: 30%;'><b>{key}</b></td>"
                
                # Format the value cell
                if key == 'performance' or key == 'confidence':
                    # Create a percentage bar
                    pct = int(value * 100)
                    bar_color = 'green' if pct > 60 else 'orange' if pct > 30 else 'red'
                    html += f"<td style='padding: 8px;'>"
                    html += f"<div style='width: 100%; background-color: #f1f1f1; border-radius: 5px;'>"
                    html += f"<div style='width: {pct}%; height: 20px; background-color: {bar_color}; border-radius: 5px;'></div>"
                    html += f"</div>{pct}%</td>"
                elif key == 'enabled':
                    status = "✅ Enabled" if value else "❌ Disabled"
                    html += f"<td style='padding: 8px;'>{status}</td>"
                else:
                    html += f"<td style='padding: 8px;'>{value}</td>"
                    
                html += "</tr>"
            
            html += "</table>"
            
            # Add resource requirements if available
            resource_reqs = self_model.get('capabilities', {}).get('resource_requirements', {})
            if capability in resource_reqs:
                html += "<h4>Resource Requirements:</h4>"
                html += "<ul>"
                for res, amount in resource_reqs[capability].items():
                    html += f"<li><b>{res}</b>: {amount}</li>"
                html += "</ul>"
            
            display(HTML(html))
    
    # Register the callback
    dropdown.observe(update_details, names='value')
    
    # Initial update
    if dropdown.options:
        dropdown.value = dropdown.options[0]
    
    # Combine widgets
    return widgets.VBox([dropdown, output])

In [ ]:
# Display capability radar chart
capability_radar = create_capability_radar_chart(self_model)
capability_radar.show()

# Display capability details explorer
capability_details = create_capability_details_ui(self_model)
display(capability_details)

## Confidence Visualization <a id="confidence"></a>

This section provides tools for visualizing the system's confidence across different knowledge domains and capabilities.

In [ ]:
def create_confidence_heatmap(self_model):
    """Create a heatmap showing confidence across different dimensions."""
    # Get self-awareness metrics
    metrics = self_model.get('metrics', {})
    dimensions = metrics.get('dimensions', {})
    
    # Get capability confidences
    capabilities = self_model.get('capabilities', {}).get('capabilities', {})
    cap_confidences = {cap: data.get('confidence', 0) for cap, data in capabilities.items()}
    
    # Combine data
    all_confidences = {
        f"Dimension: {dim}": val for dim, val in dimensions.items()
    }
    all_confidences.update({f"Capability: {cap}": val for cap, val in cap_confidences.items()})
    
    # Convert to format for heatmap
    domains = list(all_confidences.keys())
    confidences = list(all_confidences.values())
    
    # Create figure - use table format for clarity
    fig = go.Figure(data=[go.Table(
        header=dict(
            values=['Domain', 'Confidence'],
            fill_color='paleturquoise',
            align='left',
            font=dict(size=14)
        ),
        cells=dict(
            values=[domains, confidences],
            fill_color=[[f'rgba(0, {int(255*val)}, {int(255*(1-val))}, 0.7)' for val in confidences]],
            align='left',
            font=dict(size=12)
        )
    )])
    
    fig.update_layout(
        title="System Confidence Across Knowledge Domains",
        height=400 + 30 * len(domains)  # Dynamic height based on number of domains
    )
    
    return fig

def create_aggregate_confidence_gauge(self_model):
    """Create a gauge chart showing overall system confidence."""
    # Get overall confidence
    confidence = self_model.get('confidence', 0.5)
    
    # Create gauge chart
    fig = go.Figure(go.Indicator(
        mode = "gauge+number",
        value = confidence,
        title = {'text': "Overall System Confidence"},
        gauge = {
            'axis': {'range': [0, 1], 'tickwidth': 1, 'tickcolor': "darkblue"},
            'bar': {'color': "darkblue"},
            'bgcolor': "white",
            'borderwidth': 2,
            'bordercolor': "gray",
            'steps': [
                {'range': [0, 0.3], 'color': "red"},
                {'range': [0.3, 0.7], 'color': "orange"},
                {'range': [0.7, 1], 'color': "green"}
            ],
        }
    ))
    
    fig.update_layout(
        height=300,
        margin=dict(l=10, r=10, t=50, b=10)
    )
    
    return fig

In [ ]:
# Display overall confidence gauge
confidence_gauge = create_aggregate_confidence_gauge(self_model)
confidence_gauge.show()

# Display confidence heatmap
confidence_heatmap = create_confidence_heatmap(self_model)
confidence_heatmap.show()

## Temporal Evolution Tracker <a id="temporal-evolution"></a>

This section provides tools for tracking the evolution of the system's self-awareness over time.

In [ ]:
def load_historical_models(directory='../data/self_models/history/'):
    """Load historical self-models for temporal analysis."""
    historical_data = []
    
    # Create directory if it doesn't exist
    os.makedirs(directory, exist_ok=True)
    
    if not os.path.exists(directory):
        # If no history exists, create some synthetic data for demonstration
        print("No historical data found. Creating synthetic data for demonstration.")
        return create_synthetic_history()
    
    # Load all json files in the directory
    for filename in sorted(os.listdir(directory)):
        if filename.endswith('.json'):
            filepath = os.path.join(directory, filename)
            try:
                with open(filepath, 'r') as f:
                    model = json.load(f)
                    # Extract timestamp from filename or use created_at
                    timestamp = model.get('created_at', 0)
                    historical_data.append({
                        'timestamp': timestamp,
                        'model': model
                    })
            except Exception as e:
                print(f"Error loading {filepath}: {e}")
    
    # Sort by timestamp
    historical_data.sort(key=lambda x: x['timestamp'])
    
    if not historical_data:
        return create_synthetic_history()
        
    return historical_data

def create_synthetic_history():
    """Create synthetic historical data for demonstration."""
    # Start with current model
    base_model = self_model
    history = []
    
    # Create 10 synthetic historical points over the last 10 days
    import time
    current_time = time.time()
    
    for i in range(10):
        # Create a copy of the model with some variations
        model_copy = base_model.copy()
        
        # Set historical timestamp (10 days ago up to now)
        timestamp = current_time - (10-i) * 86400  # 86400 seconds per day
        model_copy['created_at'] = timestamp
        
        # Vary the metrics
        if 'metrics' in model_copy and 'dimensions' in model_copy['metrics']:
            dims = model_copy['metrics']['dimensions']
            for dim in dims:
                # Start lower and gradually increase
                base_val = dims[dim]
                dims[dim] = max(0, min(1, base_val * (0.5 + i/10)))
            
            # Also vary aggregate score
            model_copy['metrics']['aggregate_score'] = sum(dims.values()) / len(dims)
        
        # Vary confidence
        model_copy['confidence'] = 0.3 + i * 0.05
        
        history.append({
            'timestamp': timestamp,
            'model': model_copy
        })
    
    return history

def create_temporal_evolution_chart(history):
    """Create a chart showing the evolution of self-awareness metrics over time."""
    if not history:
        return go.Figure().update_layout(title="No historical data available")
    
    # Extract data for plotting
    timestamps = [datetime.fromtimestamp(entry['timestamp']) for entry in history]
    
    # Create figure
    fig = go.Figure()
    
    # Track which metrics are available
    metrics_added = set()
    
    # Add dimension metrics if available
    for i, entry in enumerate(history):
        model = entry['model']
        if 'metrics' in model and 'dimensions' in model['metrics']:
            dims = model['metrics']['dimensions']
            
            # For the first entry, set up all the metrics
            if i == 0:
                for dim, value in dims.items():
                    fig.add_trace(go.Scatter(
                        x=[timestamps[i]],
                        y=[value],
                        mode='lines+markers',
                        name=f"Dimension: {dim}"
                    ))
                    metrics_added.add(dim)
            else:
                # Update existing traces
                for j, (dim, value) in enumerate(dims.items()):
                    if dim in metrics_added:
                        fig.data[j].x = list(fig.data[j].x) + [timestamps[i]]
                        fig.data[j].y = list(fig.data[j].y) + [value]
    
    # Add aggregate score if available
    aggregate_scores = []
    for entry in history:
        model = entry['model']
        if 'metrics' in model and 'aggregate_score' in model['metrics']:
            aggregate_scores.append(model['metrics']['aggregate_score'])
        else:
            aggregate_scores.append(None)
    
    if any(score is not None for score in aggregate_scores):
        fig.add_trace(go.Scatter(
            x=timestamps,
            y=aggregate_scores,
            mode='lines+markers',
            name="Aggregate Score",
            line=dict(width=3, dash='dash')
        ))
    
    # Update layout
    fig.update_layout(
        title="Temporal Evolution of Self-Awareness Metrics",
        xaxis_title="Time",
        yaxis_title="Metric Value",
        yaxis=dict(range=[0, 1]),
        hovermode="x unified"
    )
    
    return fig

In [ ]:
# Load historical models
historical_models = load_historical_models()
print(f"Loaded {len(historical_models)} historical models")

# Create temporal evolution chart
evolution_chart = create_temporal_evolution_chart(historical_models)
evolution_chart.show()

## Anomaly Detection <a id="anomaly-detection"></a>

This section provides tools for detecting anomalies in the system's self-model and behavior.

In [ ]:
def detect_self_model_anomalies(self_model, history=None):
    """Detect anomalies in the self-model by comparing with historical patterns."""
    anomalies = []
    
    # Check for common anomaly patterns
    
    # 1. Check for unusually low confidence
    confidence = self_model.get('confidence', 0.5)
    if confidence < 0.3:
        anomalies.append({
            'type': 'low_confidence',
            'description': f"Overall system confidence is unusually low ({confidence:.2f})",
            'severity': 'high' if confidence < 0.2 else 'medium'
        })
    
    # 2. Check for capability inconsistencies
    capabilities = self_model.get('capabilities', {}).get('capabilities', {})
    for cap_id, cap_info in capabilities.items():
        perf = cap_info.get('performance', 0)
        conf = cap_info.get('confidence', 0)
        
        # Check for high performance but low confidence
        if perf > 0.7 and conf < 0.3:
            anomalies.append({
                'type': 'performance_confidence_mismatch',
                'description': f"Capability '{cap_id}' has high performance ({perf:.2f}) but low confidence ({conf:.2f})",
                'severity': 'medium'
            })
        
        # Check for low performance but high confidence
        if perf < 0.3 and conf > 0.7:
            anomalies.append({
                'type': 'performance_confidence_mismatch',
                'description': f"Capability '{cap_id}' has low performance ({perf:.2f}) but high confidence ({conf:.2f})",
                'severity': 'high'
            })
    
    # 3. Check for historical anomalies if history is provided
    if history and len(history) > 1:
        # Get metrics from last few historical points
        recent_history = history[-min(3, len(history)):]
        
        # Compare current metrics with recent history
        if 'metrics' in self_model and 'dimensions' in self_model['metrics']:
            current_dims = self_model['metrics']['dimensions']
            
            for dim, value in current_dims.items():
                # Calculate average and standard deviation from history
                historical_values = []
                for entry in recent_history:
                    if 'metrics' in entry['model'] and 'dimensions' in entry['model']['metrics']:
                        if dim in entry['model']['metrics']['dimensions']:
                            historical_values.append(entry['model']['metrics']['dimensions'][dim])
                
                if historical_values:
                    avg = sum(historical_values) / len(historical_values)
                    std_dev = (sum((x - avg) ** 2 for x in historical_values) / len(historical_values)) ** 0.5
                    
                    # Check if current value is more than 2 standard deviations away
                    if std_dev > 0 and abs(value - avg) > 2 * std_dev:
                        anomalies.append({
                            'type': 'historical_anomaly',
                            'description': f"Dimension '{dim}' value ({value:.2f}) deviates significantly from historical pattern (avg: {avg:.2f}, std: {std_dev:.2f})",
                            'severity': 'medium'
                        })
    
    # 4. Check for knowledge gap anomalies
    knowledge_boundaries = self_model.get('knowledge_boundaries', [])
    if len(knowledge_boundaries) > 5:
        anomalies.append({
            'type': 'excessive_knowledge_gaps',
            'description': f"System has an unusually high number of knowledge boundaries ({len(knowledge_boundaries)})",
            'severity': 'medium'
        })
    
    return anomalies

def display_anomaly_dashboard(self_model, history=None):
    """Create an interactive dashboard for anomaly detection and analysis."""
    # Detect anomalies
    anomalies = detect_self_model_anomalies(self_model, history)
    
    # Create HTML dashboard
    html = "<div style='background-color: #f9f9f9; padding: 15px; border-radius: 10px;'>"
    html += "<h2 style='color: #333;'>Self-Model Anomaly Dashboard</h2>"
    
    if not anomalies:
        html += "<div style='background-color: #dff0d8; padding: 10px; border-radius: 5px;'>"
        html += "<p style='color: #3c763d;'><b>✅ No anomalies detected</b> - Self-model appears consistent.</p>"
        html += "</div>"
    else:
        html += f"<p><b>{len(anomalies)} anomalies detected</b></p>"
        
        # Group by severity
        by_severity = {'high': [], 'medium': [], 'low': []}
        for anomaly in anomalies:
            by_severity[anomaly.get('severity', 'low')].append(anomaly)
        
        # Display high severity first
        if by_severity['high']:
            html += "<div style='background-color: #f2dede; padding: 10px; margin-top: 10px; border-radius: 5px;'>"
            html += f"<h3 style='color: #a94442;'>High Severity Anomalies ({len(by_severity['high'])})</h3>"
            html += "<ul>"
            for anomaly in by_severity['high']:
                html += f"<li><b>{anomaly['type']}</b>: {anomaly['description']}</li>"
            html += "</ul></div>"
        
        # Medium severity
        if by_severity['medium']:
            html += "<div style='background-color: #fcf8e3; padding: 10px; margin-top: 10px; border-radius: 5px;'>"
            html += f"<h3 style='color: #8a6d3b;'>Medium Severity Anomalies ({len(by_severity['medium'])})</h3>"
            html += "<ul>"
            for anomaly in by_severity['medium']:
                html += f"<li><b>{anomaly['type']}</b>: {anomaly['description']}</li>"
            html += "</ul></div>"
        
        # Low severity
        if by_severity['low']:
            html += "<div style='background-color: #d9edf7; padding: 10px; margin-top: 10px; border-radius: 5px;'>"
            html += f"<h3 style='color: #31708f;'>Low Severity Anomalies ({len(by_severity['low'])})</h3>"
            html += "<ul>"
            for anomaly in by_severity['low']:
                html += f"<li><b>{anomaly['type']}</b>: {anomaly['description']}</li>"
            html += "</ul></div>"
    
    html += "<div style='margin-top: 15px; font-size: 0.9em; color: #666;'>"
    html += f"<p>Last updated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}</p>"
    html += "</div>"
    
    html += "</div>"
    
    return HTML(html)

In [ ]:
# Display anomaly dashboard
anomaly_dashboard = display_anomaly_dashboard(self_model, historical_models)
display(anomaly_dashboard)

## Cross-Container Comparison <a id="cross-container"></a>

This section provides tools for comparing self-models across different containers, highlighting differences in capabilities, confidence, and self-awareness.

In [ ]:
def create_container_comparison(models):
    """Create visualizations comparing self-models across containers."""
    if not models or len(models) < 2:
        return HTML("<p>Insufficient data for cross-container comparison. Need at least 2 containers.</p>")
    
    # Extract container IDs
    container_ids = list(models.keys())
    
    # Compare metrics across dimensions
    dimension_data = {}
    for container, model in models.items():
        if 'metrics' in model and 'dimensions' in model['metrics']:
            dims = model['metrics']['dimensions']
            for dim, val in dims.items():
                if dim not in dimension_data:
                    dimension_data[dim] = {}
                dimension_data[dim][container] = val
    
    # Create comparison bar chart
    fig = go.Figure()
    
    for dim, values in dimension_data.items():
        for container in container_ids:
            if container in values:
                fig.add_trace(go.Bar(
                    x=[dim],
                    y=[values[container]],
                    name=container
                ))
    
    fig.update_layout(
        title="Self-Awareness Dimensions Across Containers",
        yaxis=dict(title="Metric Value", range=[0, 1]),
        barmode='group'
    )
    
    # Create capability comparison radar chart
    radar_fig = go.Figure()
    
    for container, model in models.items():
        capabilities = model.get('capabilities', {}).get('capabilities', {})
        if capabilities:
            cap_names = list(capabilities.keys())
            performances = [capabilities[c].get('performance', 0) for c in cap_names]
            
            radar_fig.add_trace(go.Scatterpolar(
                r=performances,
                theta=cap_names,
                fill='toself',
                name=container
            ))
    
    radar_fig.update_layout(
        title="Capability Comparison Across Containers",
        polar=dict(
            radialaxis=dict(
                visible=True,
                range=[0, 1]
            )),
        showlegend=True
    )
    
    # Create difference table
    html = "<h3>Key Differences Between Containers</h3>"
    html += "<table style='width:100%; border-collapse: collapse;'>"
    html += "<tr style='background-color: #f2f2f2;'>"
    html += "<th style='padding: 10px; text-align: left;'>Aspect</th>"
    
    for container in container_ids:
        html += f"<th style='padding: 10px; text-align: left;'>{container}</th>"
    
    html += "</tr>"
    
    # Add rows for key metrics
    aspects = [
        ('Created', lambda m: datetime.fromtimestamp(m.get('created_at', 0)).strftime('%Y-%m-%d %H:%M')),
        ('Overall Confidence', lambda m: f"{m.get('confidence', 0):.2f}"),
        ('Aggregate Score', lambda m: f"{m.get('metrics', {}).get('aggregate_score', 0):.2f}" if 'metrics' in m else 'N/A'),
        ('Knowledge Boundaries', lambda m: len(m.get('knowledge_boundaries', []))),
        ('Capabilities', lambda m: len(m.get('capabilities', {}).get('capabilities', {}))),
    ]
    
    for aspect_name, aspect_fn in aspects:
        html += "<tr style='border-bottom: 1px solid #ddd;'>"
        html += f"<td style='padding: 10px;'><b>{aspect_name}</b></td>"
        
        values = [aspect_fn(models[container]) for container in container_ids]
        all_same = all(str(val) == str(values[0]) for val in values)
        
        for val in values:
            cell_style = '' if all_same else 'background-color: #ffffe0;'
            html += f"<td style='padding: 10px; {cell_style}'>{val}</td>"
            
        html += "</tr>"
    
    html += "</table>"
    
    return fig, radar_fig, HTML(html)

In [ ]:
# Load container models
container_models = load_container_models()

# If we don't have multiple container models, create some synthetic ones for demonstration
if len(container_models) < 2:
    print("Creating synthetic container models for demonstration...")
    container_models = {
        'terminal_1': self_model,
        'terminal_2': self_model.copy()
    }
    
    # Modify terminal_2 model to show differences
    if 'metrics' in container_models['terminal_2'] and 'dimensions' in container_models['terminal_2']['metrics']:
        # Make some dimensions better, some worse
        dims = container_models['terminal_2']['metrics']['dimensions']
        for dim in dims:
            # Randomly adjust up or down
            if hash(dim) % 2 == 0:
                dims[dim] = min(1.0, dims[dim] * 1.2)
            else:
                dims[dim] = max(0.1, dims[dim] * 0.8)
    
    # Modify capabilities
    if 'capabilities' in container_models['terminal_2'] and 'capabilities' in container_models['terminal_2']['capabilities']:
        caps = container_models['terminal_2']['capabilities']['capabilities']
        for cap in caps:
            # Vary performance
            if 'performance' in caps[cap]:
                if hash(cap) % 2 == 0:
                    caps[cap]['performance'] = min(1.0, caps[cap]['performance'] * 1.3)
                else:
                    caps[cap]['performance'] = max(0.1, caps[cap]['performance'] * 0.7)

# Create container comparison visualizations
dim_fig, cap_fig, diff_table = create_container_comparison(container_models)

# Display visualizations
dim_fig.show()
cap_fig.show()
display(diff_table)

## Conclusion

This notebook provides a comprehensive set of tools for visualizing and interacting with the system's self-model. These visualizations enable researchers to better understand the system's self-knowledge, capability boundaries, confidence levels, and temporal evolution.

Key features include:
- Interactive knowledge graph for exploring the system's self-model
- Capability boundary explorer for understanding what the system can and cannot do
- Confidence visualization across knowledge domains
- Temporal evolution tracking of self-awareness metrics
- Anomaly detection in the self-model
- Cross-container comparison for parallel experimentation

This suite of tools provides a foundation for deepening our understanding of artificial self-awareness and its implications for advanced AI research.